<h1>DB connections and imports

In [1]:
import csv
import os
from mysql.connector import (connection)
from os import listdir
from os.path import isfile, join

#if os.path.exists("/home/ages/Downloads/Dwayne/10L.csv"):
#    os.remove("/home/ages/Downloads/Dwayne/30L.csv")
#else:
#    print("The file does not exist")

#pageid = input('Enter the pageid')

filelocation = "/home/ages/Downloads/Dwayne/YOLO/"
exportlocation = "/home/ages/programs/gedcomx/"


db_connection = connection.MySQLConnection(user='aviera', password='ages123',
                                  host='10.106.50.225',
                                  database='practice')

mydb = connection.MySQLConnection(user='aviera', password='ages123',
                                  host='10.106.50.225',
                                  database='practice')

mycursor = mydb.cursor(buffered=True)
mycursor2 = mydb.cursor(buffered=True)
mycursor3 = mydb.cursor(buffered=True)
mycursor4 = mydb.cursor(buffered=True)

count = 0
rw = 0
idnum = 0
xmin1 = 0.0
ymin1 = 0.0
xmax1 = 0.0
ymax1 = 0.0

print("done")

done


<h1>Delete files to start fresh

In [2]:
sql_delete1 = "DELETE FROM practice.YOLO3"
sql_delete2 = "alter table YOLO3 auto_increment=1"

# try:
#     os.remove(exportlocation+'exportedYOLOchar.csv')
# except Exception as e:
#     print(e)
# try:
#     os.remove(exportlocation+'exportedYOLObio.csv')
# except Exception as e:
#     print(e)
# try:
#     os.remove(exportlocation+'exportedYOLOname.csv')
# except Exception as e:
#     print(e)
# try:
#     os.remove(exportlocation+'exportedYOLOgen.csv')
# except Exception as e:
#     print(e)
try:
    mycursor.execute(sql_delete1,)
except Exception as e:
    print(e)
try:
    mycursor.execute(sql_delete2,)
except Exception as e:
    print(e)

print("done")

done


<h1>CSV Reader2

In [3]:
sql_insert = "INSERT INTO YOLO3 (page,generation,zone,ymin,xmin,ymax,xmax,imageid,confidence,class,name) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

onlyfiles = [f for f in listdir(filelocation)
             if isfile(join(filelocation, f))]
onlyfiles.sort()

for filename in onlyfiles:
    print(filename,'-----------------------------------------------------------------------------------------')
    with open(filelocation+filename) as csvfile:
        verfylist = csv.reader(csvfile, delimiter=',')
        for row in verfylist:
            #print(row)
            xmin1 = join(row[1])
            ymin1 = join(row[2])
            xmax1 = join(row[3])
            ymax1 = join(row[4])
            confidence1 = join(row[5])
            class1 = join(row[6])
            name1 = join(row[7])
            page1 = filename[0:8]
            zone1 = 0
            generation1 = 0
            imageid1 = page1[0:7]
            #print(count)
            val = (page1,generation1,zone1,ymin1,xmin1,ymax1,xmax1,imageid1,confidence1,class1,name1)
            mycursor.execute(sql_insert, val)
            mydb.commit()

            count = count + 1
print('done')

0001-22RROI.csv -----------------------------------------------------------------------------------------
0002-22LROI.csv -----------------------------------------------------------------------------------------
done


<h1>Updates the YOLO name = gen with generation

In [4]:
sql_YOLO3 = "SELECT * FROM practice.YOLO3 where name = 'gen' and confidence >= .7 order by id"
sql_update_YOLO3 = "update YOLO3 set generation = %s where id = %s"

mycursor4.execute(sql_YOLO3)
rowsbio3 = mycursor4.fetchall()
lenbio3 = len(rowsbio3)
gen3 = 0
#print(lenbio3)
for bioindex in range(0,lenbio3):
    rowbio3 = rowsbio3[bioindex]
    #print(bioindex)
    id3 =rowbio3[0]
    page3 = rowbio3[1]
    gen3 = gen3 + 1
    zone3 = rowbio3[3]
    ymin3 = rowbio3[4]
    xmin3 = rowbio3[5]
    ymax3 = rowbio3[6]
    xmax3 = rowbio3[7]

    val = (gen3,id3)
    #print(val)
    mycursor3.execute(sql_update_YOLO3,val)
    rowsname3 = mycursor3.fetchone()
    mydb.commit()


print("done")

done


<h1>Update the YOLO name = name with generation and zone

In [5]:
sql_YOLO3 = "SELECT * FROM practice.YOLO3 order by ymax"
sql_update_YOLO3 = "update YOLO3 set generation = %s,zone = %s where id = %s"

list1 = []   
list2 = [] 
zone = 0     
mycursor4.execute(sql_YOLO3)
rowsname3 = mycursor4.fetchall()

zone3 = 0
for rowname3 in rowsname3:
    id3 =rowname3[0]
    page3 = rowname3[1]
    gen3 = rowname3[2]
    zone3 = rowname3[3]
    ymin3 = rowname3[4]
    xmin3 = rowname3[5]
    ymax3 = rowname3[6]
    xmax3 = rowname3[7]
    confidence3 = rowname3[9]
    name3 = rowname3[11]
    #print(id3,page3,gen3,zone3)
    if name3 == 'name' and confidence3 >= .68:
        zone = zone+1
        list1.append([gen3,zone,int(ymin3),int(xmin3),int(ymax3),int(xmax3),name3,id3,confidence3])
        #print(gen3,zone,id3)
    if name3 == 'gen':
        list2.append([gen3,int(ymin3),int(xmin3),int(ymax3),int(xmax3),name3,id3])
        

for j in range(len(list2)):
    gen1,ymin1,xmin1,ymax1,xmax1,name1,id1 = list2[j]
    #print(j,gen1,zone1,ymin1,xmin1,ymax1,xmax1,name1,id1)
    
    for i in range(len(list1)):
        l1 = list1[i][2] >= ymin1-100
        l2 = list1[i][3] <= xmin1+100
        #print(i,list1[i][2],list1[i][3],list1[i][6],l1,l2)
        if l1 and l2:
            
            val = (list2[j][0],list1[i][1],list1[i][7])
            #print(val)
            mycursor3.execute(sql_update_YOLO3,val)
            mydb.commit()

#print(list1)
print("done")



done


<h1>Update the YOLO name = 'bio' with generation

In [6]:
sql_YOLO3 = "SELECT * FROM practice.YOLO3 order by ymax"
sql_update_YOLO3 = "update YOLO3 set generation = %s,zone = %s where id = %s"

list1 = []   
list2 = [] 
zone = 0     
mycursor4.execute(sql_YOLO3)
rowsname3 = mycursor4.fetchall()

zone3 = 0
for rowname3 in rowsname3:
    id3 =rowname3[0]
    page3 = rowname3[1]
    gen3 = rowname3[2]
    zone3 = rowname3[3]
    ymin3 = rowname3[4]
    xmin3 = rowname3[5]
    ymax3 = rowname3[6]
    xmax3 = rowname3[7]
    confidence3 = rowname3[9]
    name3 = rowname3[11]
    #print(id3,page3,gen3,zone3)
    if name3 == 'bio' and confidence3 >= .68:
        zone = zone+1
        list1.append([gen3,zone,int(ymin3),int(xmin3),int(ymax3),int(xmax3),name3,id3,confidence3])
        #print(gen3,zone,id3)
    if name3 == 'name':
        list2.append([gen3,int(ymin3),int(xmin3),int(ymax3),int(xmax3),name3,id3])
        

for j in range(len(list2)):
    gen1,ymin1,xmin1,ymax1,xmax1,name1,id1 = list2[j]
    #print(j,gen1,zone1,ymin1,xmin1,ymax1,xmax1,name1,id1)
    
    for i in range(len(list1)):
        l1 = list1[i][2] >= ymin1-100
        l2 = list1[i][3] <= xmin1+100
        #print(i,list1[i][2],list1[i][3],list1[i][6],l1,l2)
        if l1 and l2:
            if gen1 != 0:
                val = (list2[j][0],list1[i][1],list1[i][7])
                #print(val)
                mycursor3.execute(sql_update_YOLO3,val)
                mydb.commit()

#print(list1)
print("done")



done


<h1>Update gen and zone on characters in name regions

In [231]:
sql_char = "select * from YOLO3 order by id"
sql_update_char = "update YOLO3 set generation = %s, zone = %s where id = %s"

mycursor2.execute(sql_char)
rowschar = mycursor2.fetchall()
count = 0
counter = 0
list1 = []
list2 = []
list3 = []

for rowchar in rowschar:
    count = count + 1
    #print(count)
    id2 = rowchar[0]
    page2 = rowchar[1]
    gen2 = rowchar[2]
    zone2 = rowchar[3]
    ymin2 = rowchar[4]
    xmin2 = rowchar[5]
    ymax2 = rowchar[6]
    xmax2 = rowchar[7]
    confidence2 = rowchar[9]
    name2 = rowchar[11]
    if name2 == 'bio':
        list1.append([zone2,int(ymin2),int(xmin2),int(ymax2),int(xmax2),name2])
    if name2 == 'char' and confidence2 >= .7:
        list2.append([int(ymin2),int(xmin2),int(ymax2),int(xmax2),name2,id2])
        counter = counter + 1
        #print(counter)
    if name2 == 'gen':
        list3.append([gen2,int(ymin2),int(ymax2)])
#print(gen1,ymin1,xmin1,ymax1,xmax1,name1)
for k in range(len(list3)):
    for j in range(len(list1)):
        zone1,ymin1,xmin1,ymax1,xmax1,name1 = list1[j]
        #print("j--------------",j)
        for i in range(len(list2)):
            #print("i",i)
            l1 = list2[i][0] >= ymin1 
            l2 = list2[i][1] >= xmin1
            l3 = list2[i][2] <= ymax1
            l4 = list2[i][3] <= xmax1
            l5 = list3[k][1] >= ymin1-100
            l6 = list3[k][2] <= ymax1+100
            #print(i,l1,l2,l3,l4)
            if l1 and l2 and l3 and l4 and l5 and l6:
                #print("got here",i)
                val = (list3[k][0],list1[j][0],list2[i][5])
                print(val)
                mycursor3.execute(sql_update_char,val)
                mydb.commit()
            
#print(len(list2))
print("done")

(1, 1, 6)
(1, 1, 7)
(1, 1, 8)
(1, 1, 10)
(1, 1, 11)
(1, 1, 12)
(1, 1, 13)
(1, 1, 14)
(1, 1, 15)
(1, 1, 16)
(1, 1, 17)
(1, 1, 18)
(1, 1, 19)
(1, 1, 20)
(1, 1, 21)
(1, 1, 22)
(1, 1, 23)
(1, 1, 24)
(1, 1, 25)
(1, 1, 26)
(1, 1, 27)
(1, 1, 28)
(1, 1, 30)
(1, 1, 31)
(1, 1, 32)
(1, 1, 33)
(1, 1, 34)
(1, 1, 35)
(1, 1, 36)
(1, 1, 37)
(1, 1, 40)
(1, 1, 41)
(1, 1, 45)
(1, 1, 46)
(1, 1, 48)
(1, 1, 49)
(1, 1, 50)
(1, 1, 51)
(1, 1, 52)
(1, 1, 53)
(1, 1, 54)
(1, 1, 56)
(1, 1, 57)
(1, 1, 58)
(1, 1, 59)
(1, 1, 60)
(1, 1, 61)
(1, 1, 62)
(1, 1, 63)
(1, 1, 64)
(2, 2, 67)
(2, 2, 69)
(2, 2, 70)
(2, 2, 73)
(2, 2, 74)
(2, 2, 75)
(2, 2, 80)
(2, 2, 82)
(2, 2, 83)
(2, 2, 84)
(2, 2, 88)
(2, 2, 93)
(2, 2, 94)
(2, 2, 100)
(2, 2, 108)
(2, 2, 113)
(2, 2, 114)
(2, 2, 203)
(2, 2, 204)
(2, 2, 236)
(2, 2, 240)
(2, 2, 251)
(2, 2, 255)
(2, 2, 266)
(2, 2, 300)
(2, 3, 66)
(2, 3, 68)
(2, 3, 71)
(2, 3, 77)
(2, 3, 78)
(2, 3, 79)
(2, 3, 81)
(2, 3, 85)
(2, 3, 87)
(2, 3, 89)
(2, 3, 90)
(2, 3, 92)
(2, 3, 95)
(2, 3, 97)
(2, 3, 98)
(

<h1>Write CSV File

In [ ]:
sql_csvwrite1 = "select * from YOLO3 where confidence >= .7 and name = 'char'order by generation,zone,ymax"
sql_csvwrite2 = "select * from YOLO3 where confidence >= .7 and name = 'name'order by generation,zone,ymax"
sql_csvwrite3 = "select * from YOLO3 where confidence >= .7 and name = 'bio'order by generation,zone,ymax"
sql_csvwrite4 = "select * from YOLO3 where confidence >= .7 and name = 'gen'order by generation,zone,ymax"
mycursor.execute(sql_csvwrite1,)
mycursor2.execute(sql_csvwrite2,)
mycursor3.execute(sql_csvwrite3,)
mycursor4.execute(sql_csvwrite4,)

try:
    rows6 = mycursor.fetchall()
    fp = open(exportlocation + 'exportedYOLOchar.csv', 'w')
    myFile = csv.writer(fp)
    myFile.writerows(rows6)
    fp.close()
    mycursor.close()
except Exception as e:
    print(e)

try:
    rows7 = mycursor2.fetchall()
    fp = open(exportlocation + 'exportedYOLOname.csv', 'w')
    myFile = csv.writer(fp)
    myFile.writerows(rows7)
    fp.close()
    mycursor2.close()
except Exception as e:
    print(e)

try:
    rows8 = mycursor3.fetchall()
    fp = open(exportlocation + 'exportedYOLObio.csv', 'w')
    myFile = csv.writer(fp)
    myFile.writerows(rows8)
    fp.close()
    mycursor3.close()
except Exception as e:
    print(e)

try:
    rows9 = mycursor4.fetchall()
    fp = open(exportlocation + 'exportedYOLOgen.csv', 'w')
    myFile = csv.writer(fp)
    myFile.writerows(rows9)
    fp.close()
    mycursor4.close()
    mydb.close()
except Exception as e:
    print(e)

print("Program completed execution of all cells")
